In [189]:
import pandas as pd

In [251]:
# df = pd.read_excel('Messages/messages.xlsx')
df = pd.read_excel('Messages/labelled_messages.xlsx')

In [191]:
df = df[(df['Sentiment'] == 0) | (df['Sentiment'] == 1)]

In [192]:
df['Sentiment'].unique()

array([0, 1], dtype=int64)

In [193]:
df.reset_index(drop=True, inplace=True)
idx = 143 # index to the last manually labelled message
# idx = 60

In [194]:
df.iloc[idx]

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 143, dtype: object

In [195]:
import string
import numpy as np
def check_english(msg):
    new_msg = ''
    alpha = ' '.join(string.ascii_letters).split() 
    alpha.append(' ')
    for char in msg:
        if char in alpha:
            new_msg = new_msg + char
    msg = new_msg
    return np.nan if len(msg.split()) == 0 else msg

In [196]:
count = len(df.iloc[:idx + 1][df.iloc[:idx+1]['Message'].isna()])
idx -= count

In [197]:
df.dropna(inplace=True)

In [198]:
df.iloc[idx]

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 143, dtype: object

In [199]:
df[df['Message'].isna()]

,Message,Sentiment,Message type hint


In [200]:
# removing @mentions
df['Message'] = df['Message'].apply(lambda msg: np.nan if '@' in msg else msg)
count = len(df.iloc[:idx + 1][df.iloc[:idx+1]['Message'].isna()])
idx -= count

In [201]:
df.dropna(inplace=True)

In [202]:
df.iloc[idx]

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 143, dtype: object

In [203]:
df[df['Message'].isna()]

,Message,Sentiment,Message type hint


In [204]:
# removing # tags
df['Message'] = df['Message'].apply(lambda msg: np.nan if '#' in msg else msg)
count = len(df.iloc[:idx + 1][df.iloc[:idx+1]['Message'].isna()])
idx -= count

In [205]:
df.dropna(inplace=True)

In [206]:
df.iloc[idx]

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 143, dtype: object

In [207]:
df[df['Message'].isna()]

,Message,Sentiment,Message type hint


In [208]:
count = len(df.iloc[:idx+1][df.iloc[:idx + 1]['Message'].apply(check_english).isna()])
idx -= count
df['Message'] = df['Message'].apply(check_english)

In [209]:
df.dropna(inplace=True)

In [210]:
df.iloc[idx]

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 143, dtype: object

In [211]:
df[df['Message'].isna()]

,Message,Sentiment,Message type hint


In [212]:
df.reset_index(drop=True, inplace=True)

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Message            1071 non-null   object
 1   Sentiment          1071 non-null   int64 
 2   Message type hint  1071 non-null   object
dtypes: int64(1), object(2)
memory usage: 25.2+ KB


In [214]:
# empjis
# punctuations
# lowercase
# stopwords -> such as "and, but, or in" are removed which do not help in training 
# stemming -> running, runs are same words so converted into run as base form

In [215]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stemmer = SnowballStemmer("english", ignore_stopwords=True)
def extreme_clean(msg):
    new_msg = ''
    alpha = ' '.join(string.ascii_letters).split() 
    alpha.append(' ')
    for char in msg:
        if char in alpha:
            new_msg = new_msg + char
    msg = new_msg
    # split the message in words
    new_msg = ''
    for word in msg.split():
        new_msg = new_msg + ' ' + stemmer.stem(word)
    msg = new_msg
    nopunc = [char for char in msg if char.lower() not in string.punctuation]
    nopunc = ''.join(nopunc)
    # convert to lower case. 
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [216]:
test_msg = df.iloc[0]['Message']

In [217]:
' '.join(extreme_clean(test_msg))

'ive choke tri laugh meme partner sleep im unemploy right job let sleep scroll'

In [218]:
test_msg

'Ive been choking myself trying not to laugh at some of the memes while my partner sleeps Im unemployed right now so my job is to let him sleep while I scroll '

In [219]:
df.iloc[idx] # last message that was labbeled

Message              Yes we can and we think u are cool when not ge...
Sentiment                                                            1
Message type hint                             Positive: 0, 1: Negative
Name: 138, dtype: object

In [220]:
small_df = df.iloc[: idx + 1]

In [221]:
small_df.tail()

,Message,Sentiment,Message type hint
134,While not too hyped initially at least the Wan...,0,"Positive: 0, 1: Negative"
135,Then you have no right to call yourself a Paki...,1,"Positive: 0, 1: Negative"
136,I second Erica Taylor Sometimes its very minor...,0,"Positive: 0, 1: Negative"
137,He used up all his savings attempting to fix h...,0,"Positive: 0, 1: Negative"
138,Yes we can and we think u are cool when not ge...,1,"Positive: 0, 1: Negative"


# Final analyzer results. 
# This input will be given to CountVectorizer, then to TfIdfTransformer and then to KNeighboursClassifier

In [222]:
analyzed_messages = df['Message'].apply(extreme_clean)
for msg in analyzed_messages:
    print(msg)

['ive', 'choke', 'tri', 'laugh', 'meme', 'partner', 'sleep', 'im', 'unemploy', 'right', 'job', 'let', 'sleep', 'scroll']
['brother', 'mccarthi', 'whole', 'level', 'batshit', 'especi']
['think', 'get', 'min', 'recondit', 'wall']
['vocal', 'loudmouth', 'part', 'fanbas', 'regular', 'peopl', 'enjoy', 'stuff', 'time', 'keep', 'watch', 'dont', 'go', 'reddit', 'whine']
['bezo', 'like', 'amazon', 'zuck', 'outlier', 'vote', 'power', 'whole', 'compani', 'yeah', 'solid', 'chunk', 'ownership', 'compani', 'even', 'size', 'f']
['scene', 'iron', 'man', 'war', 'machin', 'v', 'bunch', 'robot', 'clean', 'asf']
['tip', 'deploy', 'sir']
['f', 'expens', 'work', 'absolut', 'limit', 'gymregim', 'deliv', 'physiqu', 'alter', 'result', 'im', 'sore', 'move', 'around', 'least', 'day', 'week', 'warn']
['walldouch', 'christian', 'zealot']
['frank', 'find', 'idea', 'bug', 'think', 'offens']
['dont', 'like', 'treason', 'question', 'post', 'let', 'get', 'back', 'wipe', 'everi', 'bug', 'see', 'alway', 'rememb', 'traito

# Starting to assign labels using Supervised Learning

In [223]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
X = small_df['Message']# inputs
y = small_df['Sentiment']# labelss
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=extreme_clean)),
    ('tf-idf', TfidfTransformer()),
    ('knn', KNeighborsClassifier()),
])
print(y_test[y_test==0].count())
print(y_test[y_test==1].count())


21
21


In [224]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function extreme_clean at 0x0000022C94407F60>)),
                ('tf-idf', TfidfTransformer()),
                ('knn', KNeighborsClassifier())])

In [225]:
predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[20  1]
 [16  5]]
              precision    recall  f1-score   support

           0       0.56      0.95      0.70        21
           1       0.83      0.24      0.37        21

    accuracy                           0.60        42
   macro avg       0.69      0.60      0.54        42
weighted avg       0.69      0.60      0.54        42



# Testing different K values

In [226]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_best_K(X_train_transformed, x_test_transformed, y_train, y_test):
    error = []
    precision_list = []
    recall_list = []
    f1_list = []
    
    for i in range(1, 30):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train_transformed, y_train)
        pred_i = knn.predict(x_test_transformed)
        err = np.mean(pred_i != y_test)
        
        precision = precision_score(y_test, pred_i)
        recall = recall_score(y_test, pred_i)
        f1 = f1_score(y_test, pred_i)
        
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        error.append(err)
    best = np.argmin(error) + 1 # best k 
    return best

def plot_K(error):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10,6))
    plt.plot(range(1,30),error,color='blue', linestyle='dashed', marker='o',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. K Value')
    plt.xlabel('K')
    plt.ylabel('Error Rate')

In [227]:
X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
best_K = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
# plot_K(error)
# testing data

In [228]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=extreme_clean)),
    ('tf-idf', TfidfTransformer()),
    ('knn', KNeighborsClassifier(n_neighbors=best_K)),
])
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[20  1]
 [13  8]]
              precision    recall  f1-score   support

           0       0.61      0.95      0.74        21
           1       0.89      0.38      0.53        21

    accuracy                           0.67        42
   macro avg       0.75      0.67      0.64        42
weighted avg       0.75      0.67      0.64        42



# Train on whole labelled data. Then using this data to further predict rest of the messages and assign labels.
# Using Supervised Learning Technique

In [229]:
from sklearn.pipeline import Pipeline

In [230]:
idx

138

In [231]:
small_df.groupby('Sentiment').count()

,Message,Message type hint
Sentiment,,
0,85,85
1,54,54


In [247]:
def test_accuracy(df):
    from sklearn.metrics import accuracy_score
    pipeline = Pipeline([
        ('bow', CountVectorizer(analyzer=extreme_clean)),
        ('tf-idf', TfidfTransformer()),
    ])
    
    X = df['Message']# inputs
    y = df['Sentiment']# labelss
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    pipeline.fit(X_train,y_train)
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    best_K = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
    knn = KNeighborsClassifier(n_neighbors=best_K)
    knn.fit(X_train_transformed, y_train)
    predictions = knn.predict(x_test_transformed)
    print(confusion_matrix(y_test, predictions))
    print(classification_report(y_test, predictions))
    return accuracy_score(y_test, predictions)

# Results after first iteration

In [233]:
small_df = df.iloc[:idx + 1]
large_df = df.iloc[idx + 1:]
X = small_df['Message']
y = small_df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# find k from actually labelled data

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=extreme_clean)),
    ('tf-idf', TfidfTransformer()),
])
pipeline.fit(X,y)

X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
best_K = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
knn = KNeighborsClassifier(n_neighbors=best_K)
X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
X_test = large_df['Message']
x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
knn.fit(X_train_transformed, y_train)
predictions = knn.predict(x_test_transformed) # predicting remaning using best K value
# set those to original dataframe
large_df.loc[:, 'Sentiment'] = predictions
df.iloc[idx + 1:] = large_df
test_accuracy(df)

[[214  11]
 [ 61  36]]
              precision    recall  f1-score   support

           0       0.78      0.95      0.86       225
           1       0.77      0.37      0.50        97

    accuracy                           0.78       322
   macro avg       0.77      0.66      0.68       322
weighted avg       0.77      0.78      0.75       322



0.7763975155279503

In [234]:
print(idx)

138


In [182]:
i = 1
while idx < 370:
    idx = idx + 15
    small_df = df.iloc[:idx + 1]
    large_df = df.iloc[idx + 1:]
    X = small_df['Message']
    y = small_df['Sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # find k from actually labelled data
    
    pipeline = Pipeline([
        ('bow', CountVectorizer(analyzer=extreme_clean)),
        ('tf-idf', TfidfTransformer()),
    ])
    pipeline.fit(X,y)
    
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    error = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
    knn = KNeighborsClassifier(n_neighbors=1)
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    X_test = large_df['Message']
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    knn.fit(X_train_transformed, y_train)
    predictions = knn.predict(x_test_transformed) # predicting remaning using best K value
    # set those to original dataframe
    large_df.loc[:, 'Sentiment'] = predictions
    df.iloc[idx + 1:] = large_df
    print("AT: i", i)
    test_accuracy(df)
    i += 1

AT: i 1
[[172  26]
 [ 43  81]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       198
           1       0.76      0.65      0.70       124

    accuracy                           0.79       322
   macro avg       0.78      0.76      0.77       322
weighted avg       0.78      0.79      0.78       322

AT: i 2
[[186  15]
 [ 61  60]]
              precision    recall  f1-score   support

           0       0.75      0.93      0.83       201
           1       0.80      0.50      0.61       121

    accuracy                           0.76       322
   macro avg       0.78      0.71      0.72       322
weighted avg       0.77      0.76      0.75       322

AT: i 3
[[159  27]
 [ 49  87]]
              precision    recall  f1-score   support

           0       0.76      0.85      0.81       186
           1       0.76      0.64      0.70       136

    accuracy                           0.76       322
   macro avg       0.76      0.75  

# At i = 9, the result appears good enough to stop
Good tradeoff between measures and False Positives, False Negatives

In [235]:
i = 1
while i <= 9:
    idx = idx + 15
    small_df = df.iloc[:idx + 1]
    large_df = df.iloc[idx + 1:]
    X = small_df['Message']
    y = small_df['Sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # find k from actually labelled data
    
    pipeline = Pipeline([
        ('bow', CountVectorizer(analyzer=extreme_clean)),
        ('tf-idf', TfidfTransformer()),
    ])
    pipeline.fit(X,y)
    
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    error = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
    knn = KNeighborsClassifier(n_neighbors=1)
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    X_test = large_df['Message']
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    knn.fit(X_train_transformed, y_train)
    predictions = knn.predict(x_test_transformed) # predicting remaning using best K value
    # set those to original dataframe
    large_df.loc[:, 'Sentiment'] = predictions
    df.iloc[idx + 1:] = large_df
    print("AT: i", i)
    test_accuracy(df)
    i += 1

AT: i 1
[[172  26]
 [ 43  81]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       198
           1       0.76      0.65      0.70       124

    accuracy                           0.79       322
   macro avg       0.78      0.76      0.77       322
weighted avg       0.78      0.79      0.78       322

AT: i 2
[[186  15]
 [ 61  60]]
              precision    recall  f1-score   support

           0       0.75      0.93      0.83       201
           1       0.80      0.50      0.61       121

    accuracy                           0.76       322
   macro avg       0.78      0.71      0.72       322
weighted avg       0.77      0.76      0.75       322

AT: i 3
[[159  27]
 [ 49  87]]
              precision    recall  f1-score   support

           0       0.76      0.85      0.81       186
           1       0.76      0.64      0.70       136

    accuracy                           0.76       322
   macro avg       0.76      0.75  

In [236]:
test_accuracy(df)

[[127  36]
 [ 43 116]]
              precision    recall  f1-score   support

           0       0.75      0.78      0.76       163
           1       0.76      0.73      0.75       159

    accuracy                           0.75       322
   macro avg       0.76      0.75      0.75       322
weighted avg       0.76      0.75      0.75       322



0.7546583850931677

In [238]:
new_df = df.copy()

# Futher training causes diminishing returns and does not improve performance

In [240]:
i = 1
while i <= 3:
    idx = idx + 200
    small_df = df.iloc[:idx + 1]
    large_df = df.iloc[idx + 1:]
    X = small_df['Message']
    y = small_df['Sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # find k from actually labelled data
    
    pipeline = Pipeline([
        ('bow', CountVectorizer(analyzer=extreme_clean)),
        ('tf-idf', TfidfTransformer()),
    ])
    pipeline.fit(X,y)
    
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    error = get_best_K(X_train_transformed, x_test_transformed, y_train,y_test)
    knn = KNeighborsClassifier(n_neighbors=1)
    X_train_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_train))
    X_test = large_df['Message']
    x_test_transformed = pipeline['tf-idf'].transform(pipeline['bow'].transform(X_test))
    knn.fit(X_train_transformed, y_train)
    predictions = knn.predict(x_test_transformed) # predicting remaning using best K value
    # set those to original dataframe
    large_df.loc[:, 'Sentiment'] = predictions
    df.iloc[idx + 1:] = large_df
    print("AT: i", i)
    test_accuracy(df)
    i += 1

AT: i 1
[[ 33  43]
 [ 19 227]]
              precision    recall  f1-score   support

           0       0.63      0.43      0.52        76
           1       0.84      0.92      0.88       246

    accuracy                           0.81       322
   macro avg       0.74      0.68      0.70       322
weighted avg       0.79      0.81      0.79       322

AT: i 2
[[ 32  44]
 [ 21 225]]
              precision    recall  f1-score   support

           0       0.60      0.42      0.50        76
           1       0.84      0.91      0.87       246

    accuracy                           0.80       322
   macro avg       0.72      0.67      0.68       322
weighted avg       0.78      0.80      0.78       322

AT: i 3
[[ 35  43]
 [ 21 223]]
              precision    recall  f1-score   support

           0       0.62      0.45      0.52        78
           1       0.84      0.91      0.87       244

    accuracy                           0.80       322
   macro avg       0.73      0.68  

In [241]:
df = new_df.copy()

In [252]:
test_accuracy(df)

[[127  36]
 [ 43 116]]
              precision    recall  f1-score   support

           0       0.75      0.78      0.76       163
           1       0.76      0.73      0.75       159

    accuracy                           0.75       322
   macro avg       0.76      0.75      0.75       322
weighted avg       0.76      0.75      0.75       322



0.7546583850931677

# Saving labelled messages

In [249]:
df.to_excel('Messages/labelled_messages.xlsx', index=False)